# Text Classification with `Transformer` Models

<a href="https://colab.research.google.com/drive/1ylzigj4yN0qNQjVVuVHzsrOkrq1FpDBy" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

Text classification is the process of categorizing text into predefined classes or categories based on its content. One common classification task in NLP is toxicity detection.

This is a common application of text classification in natural language processing, where the objective is to identify whether a piece of text contains language that is offensive, harmful, or threatening. To accomplish this, machine learning models are trained on large datasets of labeled toxic and non-toxic text examples and then used to predict the toxicity of new, unlabeled text.

We can define toxicity as:

> "_abusive speech targeting specific group characteristics, such as ethnic origin, religion, gender, or sexual orientation._"

For an in-depth discussion on toxicity detection as a machine learning problem, we recommend "_[Learning from the worst: Dynamically generated datasets to improve online hate detection](https://scholar.google.com/scholar_url?url=https://arxiv.org/abs/2012.15761&hl=pt-BR&sa=T&oi=gsb&ct=res&cd=0&d=7265559494033067667&ei=QUJYY6TJL4iKmgHXk5LgDQ&scisig=AAGBfm3gsyOD5eqcUPLFvWmVm8PlLcMr3g)_".

<img src="https://miro.medium.com/max/1400/1*d4k-PRw-warACDpklCh1mw.png" alt="toxic-image" width="800"/>

_Source: [Toxicity in AI Text Generation](https://towardsdatascience.com/toxicity-in-ai-text-generation-9e9d9646e68f)._

In this notebook, we will be using a dataset created from the [Toxic Comment Classification Challenge Dataset](https://github.com/tianqwang/Toxic-Comment-Classification-Challenge), created by the [Conversation AI](https://conversationai.github.io/) team, a research initiative founded by [Jigsaw](https://jigsaw.google.com/) and Google (both a part of Alphabet).

## ☣️ DISCLAIMER/WARNING ☣️

_This dataset contains text that may be considered profane, vulgar, or offensive. The vocabulary of the trained model also contains this type of language._

The original dataset contains an unequal distribution of “_hate_” and “_not hate_” samples for multi-classification. However, we created a smaller version of the original dataset and made it available on [Hugging Face](https://huggingface.co/datasets/dieineb/toxic_content_dataset).

But before starting, let us install the required libraries for this tutorial by running the cell below.

In [1]:
# Install the required libraries
%pip install datasets -q

Note: you may need to restart the kernel to use updated packages.


Now, we can download our dataset directly from the hub. 🤗

In [2]:
from datasets import load_dataset

# load the dataset from the hub
dataset = load_dataset("AiresPucrs/toxic_content_dataset")

# turn the dataset into a pandas.DataFrame
df = dataset['train'].to_pandas()

display(df.head())

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/CWLINK/.cache/huggingface/datasets/dieineb___csv/dieineb--toxic_content_dataset-6e7143093a907c22/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_text,toxic
0,explanation edits made username hardcore metal...,1
1,aww matches background colour seemingly stuck ...,1
2,hey man really trying edit war guy constantly ...,1
3,make real suggestions improvement wondered sec...,1
4,sir hero chance remember page,1


Text is one of the most widespread forms of sequence data and discrete signals (as opposed to continuous signals, like _images_ or _audio_). These sequences can be sequences of characters, syllables, words, or tokens.

Deep learning for NLP is pattern recognition applied to paragraphs, sentences, and words, just as computer vision is pattern recognition applied to videos, images, and pixels.

Like all neural networks, language models based on deep-learning architectures don't take as input raw text, i.e., you _can not multiply a word by a weight matrix, add a bias, and apply a ReLU function at the end_. Neural networks only work with numeric tensors. Thus, we need to _vectorize_ our text data, i.e., transform the text into numeric tensors.

For a comprehensive guide on how to vectorize text data, we recommend Chapter 6: Deep learning for text and sequences, in [_Deep Learning with Python_](https://tanthiamhuat.files.wordpress.com/2018/03/deeplearningwithpython.pdf). Below we will be using the `TextVectorization` layer from the [Keras](https://keras.io/) library.

We will define two hyperparameters for this class: our vocabulary size (`vocab_size`) and the maximum sequence length (`sequence_length`), which truncates a sequence after this limit is reached.

In [3]:
import tensorflow as tf

# number of words in our vocabulary
vocab_size = 20000

# sentences lengthier than 100 will be truncated
sequence_length = 100

vectorization_layer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length
    )

vectorization_layer.adapt(df.comment_text)
vocabulary = vectorization_layer.get_vocabulary()


You can save your vocabulary as a `.txt` file for later use. Don't forget to save it on your drive, or to download it before finishing this session.

If you are using Colab, run the following to save in your Drive:

```python

from google.colab import drive
drive.mount('/content/drive')

with open(r'/content/drive/MyDrive/toxic_vocabulary.txt', 'w', encoding='utf-8') as fp:
    for word in vocabulary:
        fp.write("%s\n" % word)
    fp.close()


```

☣️ DISCLAIMER - THIS VOCABULARY CONTAINS TOXIC LANGUAGE ☣️

In [4]:
with open(r'toxic_vocabulary.txt', 'w', encoding='utf-8') as fp:
    for word in vocabulary:
        fp.write("%s\n" % word)
    fp.close()


Now, we can split our dataset for training, validation, and testing (the usual ML split). For this we will use the classic `sklearn.model_selection.train_test_split`.

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

# We are doing a 80/20 split by setting `test_size=0.2`
# A constant random_state value will guarantee that the split is reproducible.
x_train, x_test, y_train, y_test = train_test_split(
    df.comment_text, df.toxic, test_size=0.2, random_state=42)

# Turn our target and labels into numpy.arrays
x_train = vectorization_layer(x_train)
y_train = np.array(y_train).astype(float)
x_test = vectorization_layer(x_test)
y_test = np.array(y_test).astype(float)

print("Training split: ", x_train.shape, y_train.shape)
print("Test split: ", x_test.shape, y_test.shape)

Training split:  (56125, 100) (56125,)
Test split:  (14032, 100) (14032,)


[Recurrent neural networks](https://en.wikipedia.org/wiki/Recurrent_neural_networks "Recurrent neural networks"), like `LSTM` and `GRU`, and convolutional networks, like `1D Convnets`, are great options for dealing with problems involving NLP. In [other notebooks](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/fa17764aa8800c388d0d298b750c686757e0861e/ML%20Explainability/NLP%20Interpreter/model_maker.ipynb) of our repository, you will see many examples of how to build these networks for tasks like sentiment analysis.

However, in this notebook, we will be using a `transformer` model, an extremely versatile and scalable architecture proposed by Vaswani et al. in [Attention Is All You Need](https://arxiv.org/abs/1706.03762).

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" alt="drawing" height="450"/>

_Source: [Attention Is All You Need](https://arxiv.org/abs/1706.03762)._

A `transformer` is a  deep learning model that uses a mechanism called [self-attention](https://en.wikipedia.org/wiki/Attention_(machine_learning)), differentially weighting the significance of each part of the input data. Like RNNs, transformers are designed to process sequential input data. However, unlike RNNs, transformers can process an entire sequence (as long as it fits on its context head) all at once (_not sequencially_). This allows for more parallelization, thus reducing training times and also allowing the training on larger datasets.

For an extremely _comprehensive_ and _ilustrated_ explanation of what is "_attention_" or how a "_transformer works_", we recommend the work of _Jay Alammar_:

- [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).
- [The Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/).

Using only the _encoder_ component of the original transformer architecture, we will implement a small transformer model in this notebook (a.k.a. a encoder-only transformer). The original transformer architecture, which consists of both an encoder and a decoder transformer block, was (originally) designed for _sequence-to-sequence_ tasks like translation.

If you are interested in learning about the transformer architecture, check our _[sequence-to-sequence](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/fa17764aa8800c388d0d298b750c686757e0861e/ML%20Intro%20Course/16_seuqnece_to_sequence.ipynb)_ tutorial.

In general, text classification can be done using the encoder component. The encoder it's a very generic module that learns to transform a sequence into a more useful representation after ingesting it.

Our model only has 4 attention heads with a capacity of 100 tokens and 4 transformer blocks. Our embedding layer's size is also restricted to embeddings with 128 dimensions and a vocabulary of 20.000 tokens (where the dense word vectors will be created).

In [6]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """
    Apply a Transformer Encoder block to the input tensor.

    Arguments:
        - inputs : tensor of shape (batch_size, sequence_length, hidden_size)
            representing the input sequence.
        - head_size : integer representing the dimensionality of the keys
            and values in each attention head.
        - num_heads : integer representing the number of attention heads.
        - ff_dim : integer representing the number of filters in the feedforward layer.
        - dropout : float between 0 and 1 representing the dropout rate.

    Returns:
        - tensor of shape (batch_size, sequence_length, hidden_size) representing
            the output of the Transformer Encoder block.
    """
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0):

    vocab_size = 20000
    sequence_length = 100
    embed_size = 128

    inputs = tf.keras.Input(shape=input_shape, dtype="int32")
    x = tf.keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=embed_size,
                              input_length=sequence_length)(inputs)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(inputs, outputs)


model = build_model(
    (x_train.shape[1]),
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.2,
)

model.compile(loss=tf.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model._name = 'ToxicityModel'
model.summary()


Model: "ToxicityModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 128)     2560000     ['input_1[0][0]']                
                                                                                                  
 layer_normalization (LayerNorm  (None, 100, 128)    256         ['embedding[0][0]']              
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 100, 128)    527488      ['layer_normalization

Below, we will train this model for one epoch using a T4 GPU (on Colab). However, you can also download a model trained on 20 epochs (+ learned vocabulary) by cloning the available repo on the hub 🤗.

```bash

!git lfs install
!git clone https://huggingface.co/dieineb/toxicity_model

```

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


callbacks = [tf.keras.callbacks.ModelCheckpoint("toxicity_model.keras",
                                                save_best_only=True),
            tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience=5,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True)]
model.fit(x_train,
          y_train,
          validation_split = 0.2,
          epochs=1,
          batch_size=16,
          verbose=1,
          callbacks=callbacks)

test_loss_score, test_acc_score = model.evaluate(x_test, y_test)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')

To clone the pre-trained model from huggingface.co, run this line on your terminal

```bash

git clone https://huggingface.co/AiresPucrs/toxicity_model

```

Or run the cell below.

In [1]:
import os

os.system("git clone https://huggingface.co/AiresPucrs/toxicity_model")
print("Cloned repo!")

Cloned repo!


To load the model, we simply create a new vectorization layer and load the saved vocabulary, and with the `tf.keras.models.load_model`, you can load the weights and architecture of any in-built model that uses the layers from the Keras API.

In [12]:
# If you cloned the model repo, the path is toxicity_model/toxicity_model.keras

toxicity_model = tf.keras.models.load_model('./toxicity_model/toxicity_model.keras')

# If you cloned the model repo, the path is toxicity_model/toxic_vocabulary.txt

with open('toxic_vocabulary.txt', encoding='utf-8') as fp:
    vocabulary = [line.strip() for line in fp]
    fp.close()

vectorization_layer = tf.keras.layers.TextVectorization(max_tokens=20000,
                                        output_mode="int",
                                        output_sequence_length=100,
                                        vocabulary=vocabulary)

strings = [
    'I think you should shut up your big mouth',
    'I do not agree with you'
]

preds = toxicity_model.predict(vectorization_layer(strings),verbose=0)

for i, string in enumerate(strings):
    print(f'{string}\n')
    print(f'Toxic 🤬 {round((1 - preds[i][0]) * 100, 2)}% | Not toxic 😊 {round(preds[i][0] * 100, 2)}\n')
    print("_" * 50)

I think you should shut up your big mouth

Toxic 🤬 95.73% | Not toxic 😊 4.27

__________________________________________________
I do not agree with you

Toxic 🤬 0.99% | Not toxic 😊 99.01

__________________________________________________


Moderation services use ML models, like the one we just made, to help make online forums safe spaces. Our model seems to discern between a "_polite reply_" and a "_rude interjection_", but a more careful analysis made reveal its vulnerabilities (something beyond the point of this tutorial).

You can also try to repurpose this architecture for other applications and tasks, like multiclass classification (e.g., by changing the function of the last layer to a `softmax` with the intended number of classes to your problem) instead of binary classification.

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).